In [1]:
import numpy as np
import pandas as pd 

In [2]:
numbers = {0:"abcefg",1:"cf", 2:"acdeg", 3:"acdfg",
          4:"bcdf",5:"abdfg",6:"abdefg",7:"acf",8:"abcdefg",9:"abcdfg"}

faux = numbers.copy()

In [3]:
egkey = {"a":'d',"b":'e',"c":'a',"d":'f',"e":'g',"f":'b',"g":'c'}

In [4]:
### part 1 
## reading in data
with open("day8.txt", "r") as f:
    datainput = []
    for x in f.readlines():
        tok = x.split("|")
        datainput.append((tok[0].split(), tok[1].split()))
count=0
for codelist, output in datainput:
    output = [len(o) for o in output]
    for o in output:
        if o in [2,4,3,7]:
            count+=1
print(count)

452


In [5]:
### part 2 functions
def builddf(numlist):
    nlist = [sorted(x) for x in numlist]
    df = pd.DataFrame(columns=['a', 'b', 'c', 'd', 'e', 'f', 'g', "num", "key"], index=list(range(10)))
    
    for i, o in enumerate(nlist):
        df.loc[i, o] = 1
        key = ("").join(o)
        df.loc[i, "key"] = key
        
    return df

class locksmith():
    def __init__(self):
        self.keys = {} ## decryption key
        self.letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
        self.codefreq = None
        self.flipcode = None
        
        numbers = {'abcefg': 0, 'cf': 1,'acdeg': 2,'acdfg': 3,
                     'bcdf': 4,'abdfg': 5,'abdefg': 6,'acf': 7,
                     'abcdefg': 8,'abcdfg': 9}
        
        nlist = list(numbers.keys())

        self.ogdf = builddf(nlist)
        
        for k,v in numbers.items():
            ix = self.ogdf[self.ogdf.key==k].index[0]
            self.ogdf.loc[ix, "num"] = v
        
        self.freqtable = self.ogdf[self.letters].sum().to_dict()
        self.flipfreq = {v:k for k,v in self.freqtable.items()}
        
        self.o7_1 = self.advanceddecrypt7p1(self.ogdf, self.freqtable)
        self.o7_1 = list(self.o7_1)[0]
        self.o7_2 = self.advanceddecrypt7p2(self.freqtable, self.o7_1)
        
    def basicdecrypt(self, code):
        basicnums = [4,6,9]
        self.codefreq = code[self.letters].sum().to_dict()
        self.flipcode = {v:k for k,v in self.codefreq.items()}
        
        for num in basicnums:
            og = self.flipfreq[num]
            cipher = self.flipcode[num]
            self.keys[og]= cipher
            
    def pairfreqix(self, n, df, freq):
        pair = []
        for k,v in freq.items():
            if v == n:
                pair.append(k)
        tmp = df[pair].dropna(axis=0, how="all")
        ix1 = tmp[pair[0]].index[tmp[pair[0]].isnull()][0]
        ix2 = tmp[pair[1]].index[tmp[pair[1]].isnull()][0]
        
        return [ix1,ix2]
    
    def advanceddecrypt7p1(self, code, freq):
        ix = self.pairfreqix(7, code, freq)
        ab = list(code.iloc[ix].key.values)
        ab.sort(key=len)
        a,b = ab
        
        return set(a).difference(b)
    
    def advanceddecrypt7p2(self, freq, p1):

        for k,v in freq.items():
            if v == 7 and k != p1:
                return k
    
    def advanceddecrypt7(self, code):
        c7_1 = self.advanceddecrypt7p1(code, self.codefreq)
        c7_1 = list(c7_1)[0]
        c7_2 = self.advanceddecrypt7p2(self.codefreq, c7_1)
        
        self.keys[self.o7_1] = c7_1
        self.keys[self.o7_2] = c7_2
        
    def freq8a(self, freq):
        num8 = []
        for k,v in freq.items():
            if v == 8:
                num8.append(k)
                
        return num8
    
    def advanceddecrypt8p1(self, df, donelist):
        
        for k in df.key.values:
            if len(k)==2:
                p1 = k
        for a in p1:
            if a not in donelist:
                return a
                
    def lastdecrypt(self, code):
        o8_1 = self.advanceddecrypt8p1(self.ogdf, list(self.keys.keys()))
        c8_1 = self.advanceddecrypt8p1(code, list(self.keys.values()))
        
        self.keys[o8_1] = c8_1
        
        o8_2 = list(set(self.letters).difference(self.keys.keys()))[0]
        c8_2 = list(set(self.letters).difference(self.keys.values()))[0]
        
        self.keys[o8_2] = c8_2
        
    def decipher(self):
        self.ogdf['key'] =self.ogdf['key'].apply(str.upper)
        newkey = {}
        for k,v in self.keys.items():
            newkey[k.upper()]=v
            
        self.ogdf['coded'] = self.ogdf['key']
        for k,v in newkey.items():
            self.ogdf['coded'] = self.ogdf['coded'].str.replace(k, v)
            
        self.ogdf['coded'] = self.ogdf['coded'].apply(sorted)
        
        cipher = self.ogdf[["coded"]].to_dict()["coded"]
        cipher = {("").join(v):k for k,v in cipher.items()}
        
        return cipher

In [6]:
total = 0
for codelist,output in datainput:
    codedf = builddf(codelist)
    smith = locksmith()
    smith.basicdecrypt(codedf)
    smith.advanceddecrypt7(codedf)
    smith.lastdecrypt(codedf)
    cipher = smith.decipher()
    output = [sorted(o) for o in output]
   
    numbah = []
    for o in output:
        code = ("").join(o)

        numbah.append(cipher[code])
    numbah = [str(x) for x in numbah] 
    numbah = ("").join(numbah)
    total+=int(numbah)
print(total)

1096964
